In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np


In [ ]:
# Directories
train_occluded_dir = '/kaggle/input/bedroom-occluded-images/train/occluded_images'
train_original_dir = '/kaggle/input/bedroom-occluded-images/train/original_images'


In [ ]:
class BedroomDataset(Dataset):
    def __init__(self, occluded_dir, original_dir, transform=None):
        self.occluded_dir = occluded_dir
        self.original_dir = original_dir
        self.transform = transform
        
        # Match occluded to original using the numeric identifiers
        self.occluded_files = sorted(os.listdir(occluded_dir), key=lambda x: int(x.split('_')[-1].split('.')[0]))
        self.original_files = sorted(os.listdir(original_dir), key=lambda x: int(x.split('_')[-1].split('.')[0]))
        
        # Ensure both directories contain matching pairs
        self.image_pairs = [(os.path.join(occluded_dir, o), os.path.join(original_dir, r)) 
                            for o, r in zip(self.occluded_files, self.original_files) 
                            if int(o.split('_')[-1].split('.')[0]) == int(r.split('_')[-1].split('.')[0])]

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        occluded_path, original_path = self.image_pairs[idx]
        
        occluded_image = Image.open(occluded_path).convert("RGB")
        original_image = Image.open(original_path).convert("RGB")

        if self.transform:
            occluded_image = self.transform(occluded_image)
            original_image = self.transform(original_image)

        return occluded_image, original_image


In [ ]:
# Define paths (update these if needed)
train_occluded_dir = "/kaggle/input/bedroom-occluded-images/train/occluded_images"
train_original_dir = "/kaggle/input/bedroom-occluded-images/train/original_images"

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load Dataset
train_dataset = BedroomDataset(train_occluded_dir, train_original_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

print(f"Total matched pairs: {len(train_dataset)}")

# Check shape of a batch
sample_occluded, sample_original = next(iter(train_loader))
print("Sample batch shape (occluded):", sample_occluded.shape)
print("Sample batch shape (original):", sample_original.shape)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        padding = kernel_size // 2
        self.conv = nn.Conv2d(input_dim + hidden_dim, hidden_dim * 4, kernel_size, padding=padding)

    def forward(self, x, hidden):
        h_cur, c_cur = hidden

        combined = torch.cat([x, h_cur], dim=1)
        conv_output = self.conv(combined)

        i, f, o, g = torch.chunk(conv_output, 4, dim=1)

        i = torch.sigmoid(i)      # Input gate
        f = torch.sigmoid(f)      # Forget gate
        o = torch.sigmoid(o)      # Output gate
        g = torch.tanh(g)         # Cell state

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next


class OptimizedPixelRNN(nn.Module):
    def __init__(self):
        super(OptimizedPixelRNN, self).__init__()

        # Encoder: Deeper for better feature extraction
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=2),  # [B, 64, 128, 128]
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # [B, 128, 64, 64]
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # [B, 256, 32, 32]
            nn.ReLU()
        )

        # ConvLSTM
        self.convlstm = ConvLSTMCell(input_dim=256, hidden_dim=256, kernel_size=3)

        # Decoder: PixelShuffle for smoother upscaling
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.PixelShuffle(2),  # [B, 128, 64, 64]
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.PixelShuffle(2),  # [B, 64, 128, 128]
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.PixelShuffle(2),  # [B, 32, 256, 256]
            nn.Conv2d(32, 3, kernel_size=3, padding=1),
            nn.Sigmoid()  # Ensure output is in range [0, 1]
        )

    def forward(self, x):
        batch_size = x.size(0)
        device = x.device

        # Encoding
        x = self.encoder(x)  # [B, 256, 32, 32]

        # Initialize hidden and cell states
        h, c = (torch.zeros(batch_size, 256, 32, 32, device=device),
                torch.zeros(batch_size, 256, 32, 32, device=device))

        # ConvLSTM step
        h, c = self.convlstm(x, (h, c))

        # Decoding
        x = self.decoder(h)  # [B, 3, 256, 256]
        return x


# Model setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = OptimizedPixelRNN().to(device)

# Dummy input to test the model
torch.cuda.empty_cache()  # Clear CUDA cache
sample_input = torch.randn(8, 3, 256, 256).to(device)

with torch.no_grad():
    output = model(sample_input)

print("Model output shape:", output.shape)  # Expected: [8, 3, 256, 256]

# Ensure no cuDNN errors due to input contiguity
print("Input contiguous:", sample_input.is_contiguous())
print("Output contiguous:", output.is_contiguous())


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from piq import ssim

# Hyperparameters
num_epochs = 20
learning_rate = 0.001
save_dir = "/kaggle/working"

# Loss and Optimizer
mse_loss = nn.MSELoss()  # Pixel-wise reconstruction loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Combined Loss Function (MSE + SSIM)
def combined_loss(outputs, targets, alpha=0.8):
    # MSE Loss (Pixel-wise accuracy)
    mse = mse_loss(outputs, targets)
    
    # SSIM Loss (Structural Similarity)
    ssim_loss = 1 - ssim(outputs, targets, data_range=1.0)
    
    # Weighted sum of both losses
    return alpha * mse + (1 - alpha) * ssim_loss

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for occluded_images, original_images in train_loader:
        occluded_images, original_images = occluded_images.to(device), original_images.to(device)

        # Forward pass
        outputs = model(occluded_images)

        # Compute combined loss (MSE + SSIM)
        loss = combined_loss(outputs, original_images)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Save model
model_path = os.path.join(save_dir, "optimized_pixelrnn.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


In [ ]:
save_dir= "/kaggle/working"
model_path = os.path.join(save_dir, "optimized_pixelrnn.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

# Path to test images
test_occluded_dir = "/kaggle/input/bedroom-occluded-images/occluded_test"

# Ensure device compatibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image Transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Custom Dataset for Occluded Test Images
class OccludedTestDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]

# Load test data
test_dataset = OccludedTestDataset(test_occluded_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Load trained model
model = OptimizedPixelRNN().to(device)
model.load_state_dict(torch.load("/kaggle/working/optimized_pixelrnn.pth", map_location=device))
model.eval()

# Evaluate and visualize
def evaluate_and_visualize(model, test_loader, device):
    model.to(device)
    model.eval()

    with torch.no_grad():
        for i, (occluded_images, filenames) in enumerate(test_loader):
            occluded_images = occluded_images.to(device)

            # Generate reconstructions
            reconstructed_images = model(occluded_images)

            # Ensure values are in valid range [0, 1]
            reconstructed_images = torch.clamp(reconstructed_images, 0, 1)

            # Move tensors to CPU for visualization
            occluded_images = occluded_images.cpu()
            reconstructed_images = reconstructed_images.cpu()

            # Display results
            for j in range(occluded_images.size(0)):
                fig, axs = plt.subplots(1, 2, figsize=(8, 4))

                axs[0].imshow(occluded_images[j].permute(1, 2, 0))
                axs[0].set_title(f"Occluded: {filenames[j]}")
                axs[0].axis("off")

                axs[1].imshow(reconstructed_images[j].permute(1, 2, 0))
                axs[1].set_title("Reconstructed")
                axs[1].axis("off")

                plt.show()

# Run evaluation
evaluate_and_visualize(model, test_loader, device)
